In [1]:
import os, sys
sys.path.append("../../")

from src.core.module import Module, Linear, LayerNorm
from src.core.losses import CrossEntropy, BCE
from src.core.optim import Standard, AdamW
from src.core.tensor import Tensor
import numpy as np
import time
from typing import List
from src.tokenizer.tokenizer import Tokenizer
import pandas as pd

In [2]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.fc1 = self.linear(7, 10, name="fc1")
        self.fc2 = self.linear(10,1, name="fc2")
        self.ln = self.layer_norm(axis=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.dropout(x, p=0.1)
        x = self.ln(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
    def train(self, x: Tensor, y: Tensor, optimizer, num_epochs=10000):
        for epoch in range(num_epochs):
            y_hat = self.forward(x)
            
            loss = BCE(y_hat, y)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            if epoch % 1 == 0:

                print(f"Epoch {epoch}, Loss: {loss.data}")

if __name__ == "__main__":
    df = pd.read_csv("../../src/experiments/data.csv")
    df['Quality'] = df['Quality'].apply(lambda x: 1 if x == "Good" else 0)
    X = Tensor(np.array(df.drop('Quality', axis=1).values))[:128]
    y = Tensor(np.array(df['Quality'].values).reshape((-1, 1)))[:128]

    X_test = Tensor(np.array(df.drop('Quality', axis=1).values))[128:]
    y_test = Tensor(np.array(df['Quality'].values).reshape((-1, 1)))[128:]

    net = Net()

    net._build(X.shape)
    optimizer = AdamW(net.parameters(), lr=0.001, clip_norm=10.0)

    net.train(X, y, optimizer)

    print(net.pipeline)


linear_0_linear_weight
linear_0_linear_bias
linear_0_linear_weight
linear_0_linear_bias
linear_0_layer_norm_gamma
linear_0_layer_norm_beta
Epoch 0, Loss: 0.7087797316425493
Epoch 1, Loss: 0.706007001233315
Epoch 2, Loss: 0.7030246235255284
Epoch 3, Loss: 0.6993305212437171
Epoch 4, Loss: 0.696759806895755
Epoch 5, Loss: 0.6940633915499095
Epoch 6, Loss: 0.6905541735187533
Epoch 7, Loss: 0.6871074424166959
Epoch 8, Loss: 0.6843422125575551
Epoch 9, Loss: 0.6810366639612137
Epoch 10, Loss: 0.6785491209068653
Epoch 11, Loss: 0.6764075190079024
Epoch 12, Loss: 0.6736272289155464
Epoch 13, Loss: 0.6700960052102942
Epoch 14, Loss: 0.6664800664402722
Epoch 15, Loss: 0.663563295859243
Epoch 16, Loss: 0.6605735113399634
Epoch 17, Loss: 0.6582226098561135
Epoch 18, Loss: 0.6538728111738155
Epoch 19, Loss: 0.6513164320250199
Epoch 20, Loss: 0.6482132236751428
Epoch 21, Loss: 0.6447439083816182
Epoch 22, Loss: 0.6422568562923159
Epoch 23, Loss: 0.6404712154307257
Epoch 24, Loss: 0.6357490399408514

KeyboardInterrupt: 